In [ ]:
# Standard library imports.
from datetime import datetime
from pathlib import Path
from re import compile, findall, sub
import sys

# Third party library imports.
import pandas as pd

# Local imports.
sys.path.insert(0,str(Path.cwd().parent/"project_scripts"))
from project_variables import bioDict
import retro_object as ro

# Create a Boston RetroObject.
boston = ro.RetroObject("bos")

# Define a function to extract the date from a game id.
def idToDate(gameId: str) -> datetime:
    pattern = compile("[A-Z]{3}(\d{4})(\d{2})(\d{2})\d")
    nums = sub(pattern, r"\1,\2,\3", gameId)
    nums = [
        int(sub("^0", "", val)) if findall("^0", val) else int(val) 
        for val in nums.split(sep = ",")
    ]
    return datetime(year = nums[0], month = nums[1], day = nums[2]).date()

# Create a Boston RetroObject. 
boston = ro.RetroObject("BOS")

# Create a pandas DataFrame for Boston's 
# July 2021 results using bevent.
bostonJulyEvents = pd.DataFrame(
    data = boston.bevent(
        yearStart = 2021,
        start = 71,
        end = 731 
    ),
    columns = [ro.beventFieldsDict[col] for col in boston.columns]
)

# Strikeouts at Boston home games, July 2021. 
strikeThree = (
    bostonJulyEvents
    .query(""" `event text*` == "K" """)
    .assign(
        date = (
            bostonJulyEvents["game id*"]
            .apply(lambda x: idToDate(x))
        ),
        batter_last_name = (
            bostonJulyEvents["res batter*"]
            .apply(lambda x: bioDict[x]["LAST"])
        ), 
        batter_first_name = (
            bostonJulyEvents["res batter*"]
            .apply(lambda x: bioDict[x]["FIRST"])
        ), 
        pitcher_last_name = (
            bostonJulyEvents["res pitcher*"]
            .apply(lambda x: bioDict[x]["LAST"])
        ),
        pitcher_first_name = (
            bostonJulyEvents["res pitcher*"]
            .apply(lambda x: bioDict[x]["FIRST"])
        )
    )
    .filter([
        "game id*", "date", "balls*", "strikes*",
        "visiting team*","batting team*",
        "batter_last_name", "batter_first_name",
        "res batter*", "res batter hand*",
        "pitcher_last_name", "pitcher_first_name",
        "res pitcher*", "res pitcher hand*"
    ])
)

# Make minor replacements in the strikeThree dataframe.
strikeThree["batting team*"] = (
    strikeThree["batting team*"]
    .replace({0: "away", 1: "home"})
)

# Redirect csv file to our export directory.
strikeThree.to_csv(Path.cwd().parent/"project_exports"/"july_strikeouts_second_export.csv")